In [ ]:
!pip install osmnx

In [ ]:
import networkx as nx
import random
import time
import matplotlib.pyplot as plt
from itertools import permutations
import osmnx as ox

In [ ]:
# Función para generar un grafo aleatorio conexo con N nodos
def generate_connected_graph(n):
    G = nx.Graph()
    nodes = list(range(n))
    G.add_nodes_from(nodes)

    # Crear un árbol aleatorio para asegurar que el grafo sea conexo
    for i in range(1, n):
        G.add_edge(i, random.randint(0, i-1), weight=random.randint(1, 10))

    # Añadir aristas adicionales aleatorias
    while G.number_of_edges() < (n * (n - 1)) // 4:
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v):
            G.add_edge(u, v, weight=random.randint(1, 10))

    return G

In [ ]:
random.seed(2115)
nodes_list = [4, 7, 10]
graphs = [generate_connected_graph(n) for n in nodes_list]

In [ ]:
def plot_multiple_graphs(graphs, titles, figsize=(15, 5)):
    fig, axs = plt.subplots(1, len(graphs), figsize=figsize)  # Crear subplots
    for i, (G, title) in enumerate(zip(graphs, titles)):
        ax = axs[i]
        nx.draw(G, with_labels=True, node_color="lightblue", node_size=500, font_size=10, font_weight='bold', ax=ax)
        ax.set_title(title)
    plt.show()

In [ ]:
# Visualizar los grafos
titles = ["Grafo con 4 nodos", "Grafo con 7 nodos", "Grafo con 10 nodos"]
plot_multiple_graphs(graphs, titles)

In [ ]:
G = nx.erdos_renyi_graph(7, 0.5, seed=2115, directed=False)
plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold')
plt.title("Grafo")
plt.show()

In [ ]:
# Función de fuerza bruta para calcular ruta con menor costo que pasa por todos los nodos
def brute_force_tsp(G):
    nodes = list(G.nodes)
    all_routes = permutations(nodes)
    min_cost = float('inf')
    best_route = None
    for route in all_routes:
        try:
            cost = sum(G[route[i]][route[i+1]]['weight'] for i in range(len(route) - 1))
            if cost < min_cost:
                min_cost = cost
                best_route = route
        except KeyError: # Ignorar rutas que no sean posibles debido a la falta de aristas
            continue
    return best_route, min_cost

In [ ]:
list(permutations(graphs[1].nodes()))

In [ ]:
# Función para calcular tiempo de ejecución
def calculate_execution_times(G):
    start_time = time.time()
    route, cost = brute_force_tsp(G)
    excecution_time = time.time() - start_time

    return route, cost, excecution_time

In [ ]:
routes = []
costs = []
times = []
for G in graphs:
    n = G.number_of_nodes()
    route, cost, excecution_time = calculate_execution_times(G)
    routes.append(route)
    costs.append(cost)
    times.append(excecution_time)
    print(f"Número de nodos: {n}")

# Graficar los tiempos de ejecución
plt.figure(figsize=(8, 6))
plt.plot(nodes_list, times, marker='o')
plt.xlabel('Numero de nodos')
plt.ylabel('Tiempo de ejecucion (seconds)')
plt.title('Tiempo de ejecucion TSP fuerza bruta')
plt.grid(True)
plt.show()

In [ ]:
# Mostrar los costos de ambas soluciones
for i, n in enumerate(nodes_list):
    print(f"Número de nodos: {n}")
    print(f"Costo de la ruta con fuerza bruta: {costs[i]}")
    print("-" * 40)


In [ ]:
def plot_multiple_graphs_with_routes(graphs, routes, titles, figsize=(15, 5)):
    fig, axs = plt.subplots(1, len(graphs), figsize=figsize)  # Crear subplots para varios grafos
    for i, (G, title) in enumerate(zip(graphs, titles)):
        ax = axs[i] if len(graphs) > 1 else axs  # Obtener el subplot correspondiente
        pos = nx.spring_layout(G)  # Layout para graficar el grafo

        # Dibujar el grafo
        nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10, font_weight='bold', ax=ax)

        # Dibujar la ruta de fuerza bruta en rojo
        edges_brute_force = [(routes[i][j], routes[i][j+1]) for j in range(len(routes[i]) - 1)]
        nx.draw_networkx_edges(G, pos, edgelist=edges_brute_force, edge_color='r', width=2, ax=ax, label="Brute Force Route")

        ax.set_title(title)
        ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
titles = ["Grafo con 4 nodos", "Grafo con 7 nodos", "Grafo con 10 nodos"]
plot_multiple_graphs_with_routes(graphs, routes, titles)

# Usando redes reales

In [ ]:
# Descargar la red vial de Macul (Santiago, Chile)
place_name = "Macul, Santiago, Chile"
G = ox.graph_from_place(place_name, network_type="drive")

# Convertir la red vial a un grafo no dirigido (por si tiene direcciones)
G_undirected = G.to_undirected()

In [ ]:
# 1. Calcular la centralidad de grado
degree_centrality = nx.degree_centrality(G_undirected)

# 2. Calcular la centralidad de cercanía
closeness_centrality = nx.closeness_centrality(G_undirected)

# 3. Calcular la centralidad de intermediación
betweenness_centrality = nx.betweenness_centrality(G_undirected, normalized=True)

# Mostrar algunas métricas de ejemplo para los primeros 5 nodos
for i, node in enumerate(G_undirected.nodes):
    if i == 5:  # Solo mostramos los primeros 5 nodos
        break
    print(f"Nodo: {node}")
    print(f"  Centralidad de Grado: {degree_centrality[node]:.4f}")
    print(f"  Centralidad de Cercanía: {closeness_centrality[node]:.4f}")
    print(f"  Centralidad de Intermediación: {betweenness_centrality[node]:.4f}")
    print()

In [ ]:
# Añadir las métricas de centralidad como atributos a los nodos para su visualización
nx.set_node_attributes(G_undirected, degree_centrality, "degree_centrality")
nx.set_node_attributes(G_undirected, closeness_centrality, "closeness_centrality")
nx.set_node_attributes(G_undirected, betweenness_centrality, "betweenness_centrality")

In [ ]:
def plot_graph_with_centrality(G, centrality_attr, cmap, title):
    # Obtener los valores de centralidad
    centrality_values = nx.get_node_attributes(G, centrality_attr)
    norm = plt.Normalize(vmin=min(centrality_values.values()), vmax=max(centrality_values.values()))  # Normalizar los valores

    # Obtener los colores para cada nodo según la centralidad
    nc = ox.plot.get_node_colors_by_attr(G, centrality_attr, cmap=cmap)

    # Graficar el grafo y capturar fig y ax
    fig, ax = ox.plot_graph(
        G,
        node_color=nc,
        node_size=20,
        edge_linewidth=0.5,
        bgcolor="white",
        show=False,
        close=False
    )

    # Agregar la barra de color
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(centrality_attr.replace('_', ' ').title(), rotation=270, labelpad=20)  # Etiqueta para la barra de colores

    # Agregar el título
    ax.set_title(title, fontsize=16)

    plt.show()


In [ ]:
plot_graph_with_centrality(G_undirected, "degree_centrality", cmap="plasma", title="Grafo con Centralidad de Grado")

In [ ]:
plot_graph_with_centrality(G_undirected, "closeness_centrality", cmap="viridis", title="Grafo con Centralidad de Cercanía")

In [ ]:
plot_graph_with_centrality(G_undirected, "betweenness_centrality", cmap="inferno", title="Grafo con Centralidad de Intermediación")